# 🚀 Khmer OCR Hyperparameter Tuning on Google Colab

**Phase 3.1: Systematic Hyperparameter Optimization**

This notebook runs hyperparameter tuning experiments for the Khmer digits OCR model with GPU acceleration.

## 📋 Setup Checklist:
1. ✅ Enable GPU: Runtime → Change runtime type → GPU
2. ✅ Upload your project files (or clone from GitHub)
3. ✅ Run all cells in order
4. ✅ Download results when complete


## 🔧 Environment Setup

In [1]:
# Check GPU availability
import torch
import sys
print(f"Python version: {sys.version}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️ No GPU detected. Enable GPU in Runtime → Change runtime type")

Python version: 3.10.12 | packaged by conda-forge | (main, Jun 23 2023, 22:39:40) [Clang 15.0.7 ]
PyTorch version: 2.2.2
CUDA available: False
⚠️ No GPU detected. Enable GPU in Runtime → Change runtime type


In [ ]:
# Install additional dependencies if needed
!pip install -q opencv-python-headless
!pip install -q albumentations
!pip install -q tensorboard
!pip install -q efficientnet-pytorch
!pip install -q Pillow
!pip install -q pyyaml
!pip install -q tqdm

print("✅ Dependencies installed!")

## 📁 Upload Project Files

**Option 1: Upload ZIP file**
- Compress your entire project folder
- Upload and extract using the cell below

**Option 2: Clone from GitHub** (if your project is on GitHub)
- Use the git clone cell below

In [ ]:
# Option 1: Upload ZIP file
# from google.colab import files
# import zipfile
# import os

# print("📁 Upload your project ZIP file:")
# uploaded = files.upload()

# # Extract the uploaded ZIP
# for filename in uploaded.keys():
#     if filename.endswith('.zip'):
#         print(f"Extracting {filename}...")
#         with zipfile.ZipFile(filename, 'r') as zip_ref:
#             zip_ref.extractall('.')
#         print(f"✅ Extracted {filename}")
        
# # List contents to verify
# print("\n📂 Current directory contents:")
# !ls -la

In [ ]:
# Option 2: Clone from GitHub (uncomment and modify URL)
!git clone https://github.com/yourusername/khmer-ocr-digits.git
%cd khmer-ocr-digits
print("✅ Project cloned from GitHub")

In [2]:
# Setup project paths
import os
import sys
from pathlib import Path

# Find project root (adjust path if needed)
project_root = None
for root in ['.', './khmer-ocr-digits', '../']:
    if os.path.exists(os.path.join(root, 'src')):
        project_root = Path(root).resolve()
        break

if project_root:
    os.chdir(project_root)
    sys.path.append(str(project_root / 'src'))
    print(f"✅ Project root: {project_root}")
    print(f"✅ Added to Python path: {project_root / 'src'}")
else:
    print("❌ Could not find project root. Please check your upload.")
    print("Current directory contents:")
    !ls -la

✅ Project root: /Users/kunthet/dev/khm/khmer-ocr-digits
✅ Added to Python path: /Users/kunthet/dev/khm/khmer-ocr-digits/src


## ✅ Verify Setup

In [3]:
# Test imports to verify everything is working
try:
    from modules.data_utils import KhmerDigitsDataset
    from models import create_model
    from modules.trainers import OCRTrainer
    from modules.trainers.utils import setup_training_environment, TrainingConfig
    print("✅ All imports successful!")
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("Please check your project structure and file uploads.")

✅ All imports successful!


In [4]:
# Check if data files exist
import os

required_files = [
    'generated_data/metadata.yaml',
    'config/phase3_training_configs.yaml',
    'config/model_config.yaml'
]

print("📋 Checking required files:")
all_files_exist = True
for file_path in required_files:
    if os.path.exists(file_path):
        print(f"✅ {file_path}")
    else:
        print(f"❌ {file_path} - NOT FOUND")
        all_files_exist = False

if all_files_exist:
    print("\n🎉 All required files found! Ready to start training.")
else:
    print("\n⚠️ Some files are missing. Please check your upload.")

📋 Checking required files:
✅ generated_data/metadata.yaml
✅ config/phase3_training_configs.yaml
✅ config/model_config.yaml

🎉 All required files found! Ready to start training.


## 🚀 Hyperparameter Tuning

In [5]:
# Colab-adapted Hyperparameter Tuning Class
import yaml
import json
import logging
import time
from datetime import datetime
from typing import Dict, List, Any
import torch
from IPython.display import display, HTML, clear_output
import matplotlib.pyplot as plt

class ColabHyperparameterTuner:
    """Colab-optimized hyperparameter tuner with live progress tracking."""
    
    def __init__(self, config_file: str = "config/phase3_training_configs.yaml"):
        self.config_file = config_file
        self.results = []
        self.best_result = None
        self.experiments_completed = 0
        
        # Load configuration
        with open(config_file, 'r') as f:
            self.config = yaml.safe_load(f)
        
        # Setup logging for Colab
        self.setup_colab_logging()
        
    def setup_colab_logging(self):
        """Setup logging optimized for Colab."""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        # Configure logging
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s',
            force=True
        )
        self.logger = logging.getLogger(__name__)
        
    def create_training_config(self, experiment_config: Dict) -> TrainingConfig:
        """Create TrainingConfig object from experiment configuration."""
        # Adapt for Colab GPU environment
        config = TrainingConfig(
            experiment_name=experiment_config['experiment_name'],
            model_name=experiment_config['model']['name'],
            model_config_path=experiment_config['model']['config_path'],
            metadata_path=experiment_config['data']['metadata_path'],
            batch_size=experiment_config['training']['batch_size'],
            num_workers=2,  # Reduced for Colab
            pin_memory=True,  # Enable for GPU
            learning_rate=experiment_config['training']['learning_rate'],
            weight_decay=experiment_config['training']['weight_decay'],
            num_epochs=experiment_config['training']['num_epochs'],
            device="auto",  # Will auto-detect GPU
            mixed_precision=True,  # Enable for GPU speedup
            gradient_clip_norm=experiment_config['training']['gradient_clip_norm'],
            loss_type=experiment_config['training']['loss_type'],
            label_smoothing=experiment_config['training'].get('label_smoothing', 0.0),
            scheduler_type=experiment_config['scheduler']['type'],
            step_size=experiment_config['scheduler'].get('step_size', 10),
            gamma=experiment_config['scheduler'].get('gamma', 0.5),
            early_stopping_patience=experiment_config['early_stopping']['patience'],
            early_stopping_min_delta=experiment_config['early_stopping']['min_delta'],
            log_every_n_steps=experiment_config['training']['log_every_n_steps'],
            save_every_n_epochs=experiment_config['training']['save_every_n_epochs'],
            keep_n_checkpoints=2,  # Reduced for Colab storage
            use_tensorboard=True,
            output_dir="training_output"
        )
        return config

    def display_progress(self, current_exp: int, total_exp: int, exp_name: str, status: str):
        """Display live progress in Colab."""
        progress_html = f"""
        <div style="border: 2px solid #4CAF50; padding: 10px; margin: 10px 0; border-radius: 5px;">
            <h3>🚀 Hyperparameter Tuning Progress</h3>
            <p><strong>Experiment:</strong> {current_exp}/{total_exp} - {exp_name}</p>
            <p><strong>Status:</strong> {status}</p>
            <div style="background-color: #f0f0f0; border-radius: 10px; padding: 3px;">
                <div style="background-color: #4CAF50; height: 20px; border-radius: 10px; width: {(current_exp/total_exp)*100}%;"></div>
            </div>
            <p>{(current_exp/total_exp)*100:.1f}% Complete</p>
        </div>
        """
        display(HTML(progress_html))

    def run_single_experiment(self, experiment_name: str, experiment_config: Dict) -> Dict:
        """Run a single hyperparameter experiment with Colab optimizations."""
        self.logger.info(f"🚀 Starting experiment: {experiment_name}")
        
        start_time = time.time()
        
        try:
            # Create training configuration
            training_config = self.create_training_config(experiment_config)
            
            # Setup environment
            env_info = setup_training_environment(training_config)
            device = env_info['device']
            exp_dir = env_info['dirs']['experiment_dir']
            
            self.logger.info(f"🖥️ Using device: {device}")
            
            # Load datasets
            self.logger.info("📚 Loading datasets...")
            
            from modules.data_utils.preprocessing import get_train_transforms, get_val_transforms
            
            train_dataset = KhmerDigitsDataset(
                metadata_path=training_config.metadata_path,
                split='train',
                transform=get_train_transforms()
            )
            val_dataset = KhmerDigitsDataset(
                metadata_path=training_config.metadata_path,
                split='val',
                transform=get_val_transforms()
            )
            
            # Create data loaders
            from torch.utils.data import DataLoader
            from modules.data_utils.dataset import collate_fn
            
            train_loader = DataLoader(
                train_dataset,
                batch_size=training_config.batch_size,
                shuffle=True,
                num_workers=training_config.num_workers,
                pin_memory=training_config.pin_memory,
                collate_fn=collate_fn
            )
            
            val_loader = DataLoader(
                val_dataset,
                batch_size=training_config.batch_size,
                shuffle=False,
                num_workers=training_config.num_workers,
                pin_memory=training_config.pin_memory,
                collate_fn=collate_fn
            )
            
            self.logger.info(f"📊 Training samples: {len(train_dataset)}")
            self.logger.info(f"📊 Validation samples: {len(val_dataset)}")
            
            # Create model
            self.logger.info("🏗️ Creating model...")
            model = create_model(
                preset=training_config.model_name,
                vocab_size=len(train_dataset.char_to_idx),
                max_sequence_length=train_dataset.max_sequence_length + 1
            )
            
            # Initialize trainer
            trainer = OCRTrainer(
                model=model,
                train_loader=train_loader,
                val_loader=val_loader,
                config=training_config,
                device=device
            )
            
            # Run training
            self.logger.info("🎯 Starting training...")
            training_history = trainer.train()
            
            # Calculate metrics
            end_time = time.time()
            training_time = end_time - start_time
            
            # Extract metrics correctly from training history
            training_hist = training_history.get('training_history', {})
            val_metrics_list = training_hist.get('val_metrics', [])
            
            # Calculate best metrics across all epochs
            best_char_acc = 0.0
            best_seq_acc = 0.0
            
            if val_metrics_list:
                char_accuracies = [m.get('char_accuracy', 0.0) for m in val_metrics_list]
                seq_accuracies = [m.get('seq_accuracy', 0.0) for m in val_metrics_list]
                best_char_acc = max(char_accuracies) if char_accuracies else 0.0
                best_seq_acc = max(seq_accuracies) if seq_accuracies else 0.0
            
            # Get final loss
            train_losses = training_hist.get('train_loss', [])
            final_train_loss = train_losses[-1] if train_losses else float('inf')
            
            # Create result
            result = {
                'experiment_name': experiment_name,
                'status': 'completed',
                'training_time': training_time,
                'best_val_char_accuracy': best_char_acc,
                'best_val_seq_accuracy': best_seq_acc,
                'final_train_loss': final_train_loss,
                'device': str(device),
                'hyperparameters': {
                    'model_name': experiment_config['model']['name'],
                    'batch_size': experiment_config['training']['batch_size'],
                    'learning_rate': experiment_config['training']['learning_rate'],
                    'weight_decay': experiment_config['training']['weight_decay'],
                    'loss_type': experiment_config['training']['loss_type'],
                    'scheduler_type': experiment_config['scheduler']['type']
                },
                'training_history': training_hist
            }
            
            self.logger.info(f"✅ Experiment {experiment_name} completed!")
            self.logger.info(f"🏆 Best character accuracy: {result['best_val_char_accuracy']:.4f}")
            self.logger.info(f"🏆 Best sequence accuracy: {result['best_val_seq_accuracy']:.4f}")
            
            return result
            
        except Exception as e:
            self.logger.error(f"❌ Experiment {experiment_name} failed: {str(e)}")
            return {
                'experiment_name': experiment_name,
                'status': 'failed',
                'error': str(e),
                'training_time': time.time() - start_time
            }

    def run_experiments(self, experiment_names: List[str] = None):
        """Run all or specified experiments with Colab progress tracking."""
        experiments = self.config['experiments']
        
        if experiment_names:
            experiments = {name: config for name, config in experiments.items() 
                          if name in experiment_names}
        
        total_experiments = len(experiments)
        self.logger.info(f"🚀 Starting hyperparameter tuning with {total_experiments} experiments")
        
        for i, (exp_name, exp_config) in enumerate(experiments.items(), 1):
            # Update progress display
            clear_output(wait=True)
            self.display_progress(i-1, total_experiments, exp_name, "Starting...")
            
            result = self.run_single_experiment(exp_name, exp_config)
            self.results.append(result)
            
            # Update best result
            if (result.get('status') == 'completed' and 
                (self.best_result is None or 
                 result['best_val_char_accuracy'] > 
                 self.best_result['best_val_char_accuracy'])):
                self.best_result = result
            
            self.experiments_completed += 1
            
            # Update progress display
            status = "✅ Completed" if result.get('status') == 'completed' else "❌ Failed"
            self.display_progress(i, total_experiments, exp_name, status)
            
            # Display current results
            self.display_results_table()

    def display_results_table(self):
        """Display results in a nice HTML table."""
        if not self.results:
            return
            
        # Sort results by character accuracy
        completed_results = [r for r in self.results if r.get('status') == 'completed']
        completed_results.sort(key=lambda x: x.get('best_val_char_accuracy', 0), reverse=True)
        
        html = """
        <div style="margin: 20px 0;">
            <h3>📊 Current Results</h3>
            <table style="border-collapse: collapse; width: 100%; border: 1px solid #ddd;">
                <thead>
                    <tr style="background-color: #f2f2f2;">
                        <th style="border: 1px solid #ddd; padding: 8px;">Rank</th>
                        <th style="border: 1px solid #ddd; padding: 8px;">Experiment</th>
                        <th style="border: 1px solid #ddd; padding: 8px;">Char Acc</th>
                        <th style="border: 1px solid #ddd; padding: 8px;">Seq Acc</th>
                        <th style="border: 1px solid #ddd; padding: 8px;">Model</th>
                        <th style="border: 1px solid #ddd; padding: 8px;">Status</th>
                    </tr>
                </thead>
                <tbody>
        """
        
        for i, result in enumerate(completed_results, 1):
            status_emoji = "✅" if result.get('status') == 'completed' else "❌"
            char_acc = result.get('best_val_char_accuracy', 0) * 100
            seq_acc = result.get('best_val_seq_accuracy', 0) * 100
            model_name = result.get('hyperparameters', {}).get('model_name', 'unknown')
            
            html += f"""
                <tr>
                    <td style="border: 1px solid #ddd; padding: 8px; text-align: center;">{i}</td>
                    <td style="border: 1px solid #ddd; padding: 8px;">{result['experiment_name']}</td>
                    <td style="border: 1px solid #ddd; padding: 8px; text-align: center;">{char_acc:.1f}%</td>
                    <td style="border: 1px solid #ddd; padding: 8px; text-align: center;">{seq_acc:.1f}%</td>
                    <td style="border: 1px solid #ddd; padding: 8px; text-align: center;">{model_name}</td>
                    <td style="border: 1px solid #ddd; padding: 8px; text-align: center;">{status_emoji}</td>
                </tr>
            """
        
        # Add failed experiments
        failed_results = [r for r in self.results if r.get('status') == 'failed']
        for result in failed_results:
            html += f"""
                <tr style="background-color: #ffe6e6;">
                    <td style="border: 1px solid #ddd; padding: 8px; text-align: center;">-</td>
                    <td style="border: 1px solid #ddd; padding: 8px;">{result['experiment_name']}</td>
                    <td style="border: 1px solid #ddd; padding: 8px; text-align: center;">FAILED</td>
                    <td style="border: 1px solid #ddd; padding: 8px; text-align: center;">FAILED</td>
                    <td style="border: 1px solid #ddd; padding: 8px; text-align: center;">-</td>
                    <td style="border: 1px solid #ddd; padding: 8px; text-align: center;">❌</td>
                </tr>
            """
        
        html += """
                </tbody>
            </table>
        </div>
        """
        
        display(HTML(html))

    def save_results(self):
        """Save tuning results to file and prepare for download."""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        results_file = f"colab_hyperparameter_results_{timestamp}.json"
        
        with open(results_file, 'w') as f:
            json.dump({
                'timestamp': timestamp,
                'platform': 'Google Colab',
                'gpu_info': torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU',
                'best_result': self.best_result,
                'all_results': self.results
            }, f, indent=2)
        
        self.logger.info(f"💾 Results saved to {results_file}")
        return results_file

print("✅ Colab Hyperparameter Tuner class loaded!")

✅ Colab Hyperparameter Tuner class loaded!


## 🧪 Quick Test (Single Experiment)

Test with just one experiment to make sure everything works:

In [6]:
# Quick test with the best performing configuration
print("🧪 Running quick test with conservative_small configuration...")

# Initialize tuner
tuner = ColabHyperparameterTuner()

# Run just the best configuration for testing
tuner.run_experiments(['conservative_small'])

print("\n✅ Quick test completed!")
if tuner.best_result:
    print(f"🏆 Result: {tuner.best_result['best_val_char_accuracy']:.1%} character accuracy")

2025-06-24 12:38:53,637 - INFO - 🚀 Starting experiment: conservative_small
2025-06-24 12:38:53,648 - INFO - Training environment setup complete
2025-06-24 12:38:53,650 - INFO - Experiment directory: training_output/conservative_small
2025-06-24 12:38:53,651 - INFO - Device: cpu
2025-06-24 12:38:53,652 - INFO - Mixed precision: True
2025-06-24 12:38:53,653 - INFO - 🖥️ Using device: cpu
2025-06-24 12:38:53,654 - INFO - 📚 Loading datasets...


Loaded 4000 samples for split 'train'


2025-06-24 12:39:11,660 - INFO - 📊 Training samples: 4000
2025-06-24 12:39:11,661 - INFO - 📊 Validation samples: 1000
2025-06-24 12:39:11,662 - INFO - 🏗️ Creating model...


Loaded 1000 samples for split 'val'


2025-06-24 12:39:12,099 - INFO - Model parameters: {'total_parameters': 12484789, 'trainable_parameters': 12484789, 'non_trainable_parameters': 0}
2025-06-24 12:39:12,100 - INFO - OCR Trainer initialized
2025-06-24 12:39:12,102 - INFO - Vocabulary size: 13
2025-06-24 12:39:12,103 - INFO - Characters: ['U+17E0', 'U+17E1', 'U+17E2', 'U+17E3', 'U+17E4', 'U+17E5', 'U+17E6', 'U+17E7', 'U+17E8', 'U+17E9', '<EOS>', '<PAD>', '<BLANK>']
2025-06-24 12:39:12,104 - INFO - 🎯 Starting training...
2025-06-24 12:39:12,106 - INFO - Starting training for 2 epochs
2025-06-24 12:39:12,107 - INFO - Training samples: 4000
2025-06-24 12:39:12,109 - INFO - Validation samples: 1000
2025-06-24 12:39:27,563 - INFO - Epoch 1 Batch 0 - Images: torch.Size([32, 3, 128, 64]), Labels: torch.Size([32, 9]), Predictions: torch.Size([32, 9, 13])
2025-06-24 12:39:28,043 - INFO - Epoch   1 | Batch    0/125 | Loss: 2.8828 | LR: 0.001000
2025-06-24 12:39:28,216 - INFO - Epoch 1 Batch 1 - Images: torch.Size([32, 3, 128, 64]), 

Rank,Experiment,Char Acc,Seq Acc,Model,Status
1,conservative_small,29.0%,4.9%,small,✅



✅ Quick test completed!
🏆 Result: 29.0% character accuracy


## 🚀 Full Hyperparameter Tuning

Run all experiments (this will take several hours with GPU):

In [7]:
# Run all hyperparameter experiments
print("🚀 Starting full hyperparameter tuning...")
print("⏰ This will take several hours with GPU acceleration.")
print("💡 You can stop and resume by running specific experiments.")

# Initialize fresh tuner
full_tuner = ColabHyperparameterTuner()

# Run all experiments
full_tuner.run_experiments()

# Save results
results_file = full_tuner.save_results()

print(f"\n🎉 Hyperparameter tuning completed!")
if full_tuner.best_result:
    print(f"🏆 Best result: {full_tuner.best_result['experiment_name']}")
    print(f"📊 Character accuracy: {full_tuner.best_result['best_val_char_accuracy']:.1%}")
    print(f"📊 Sequence accuracy: {full_tuner.best_result['best_val_seq_accuracy']:.1%}")

2025-06-24 13:20:54,502 - INFO - 🚀 Starting experiment: fast_convergence
2025-06-24 13:20:54,516 - INFO - Training environment setup complete
2025-06-24 13:20:54,516 - INFO - Experiment directory: training_output/fast_convergence
2025-06-24 13:20:54,517 - INFO - Device: cpu
2025-06-24 13:20:54,518 - INFO - Mixed precision: True
2025-06-24 13:20:54,519 - INFO - 🖥️ Using device: cpu
2025-06-24 13:20:54,519 - INFO - 📚 Loading datasets...


Loaded 4000 samples for split 'train'


2025-06-24 13:21:06,333 - INFO - 📊 Training samples: 4000
2025-06-24 13:21:06,334 - INFO - 📊 Validation samples: 1000
2025-06-24 13:21:06,335 - INFO - 🏗️ Creating model...


Loaded 1000 samples for split 'val'


2025-06-24 13:21:06,583 - INFO - Model parameters: {'total_parameters': 16167733, 'trainable_parameters': 16167733, 'non_trainable_parameters': 0}
2025-06-24 13:21:06,584 - INFO - OCR Trainer initialized
2025-06-24 13:21:06,584 - INFO - Vocabulary size: 13
2025-06-24 13:21:06,584 - INFO - Characters: ['U+17E0', 'U+17E1', 'U+17E2', 'U+17E3', 'U+17E4', 'U+17E5', 'U+17E6', 'U+17E7', 'U+17E8', 'U+17E9', '<EOS>', '<PAD>', '<BLANK>']
2025-06-24 13:21:06,585 - INFO - 🎯 Starting training...
2025-06-24 13:21:06,586 - INFO - Starting training for 2 epochs
2025-06-24 13:21:06,586 - INFO - Training samples: 4000
2025-06-24 13:21:06,587 - INFO - Validation samples: 1000
2025-06-24 13:21:17,999 - INFO - Epoch 1 Batch 0 - Images: torch.Size([96, 3, 128, 64]), Labels: torch.Size([96, 9]), Predictions: torch.Size([96, 9, 13])
2025-06-24 13:21:19,297 - INFO - Epoch   1 | Batch    0/42 | Loss: 2.9399 | LR: 0.005000
2025-06-24 13:21:19,884 - INFO - Epoch 1 Batch 1 - Images: torch.Size([96, 3, 128, 64]), L

Rank,Experiment,Char Acc,Seq Acc,Model,Status
1,conservative_small,29.3%,5.4%,small,✅
2,focal_loss_experiment,25.3%,0.1%,medium,✅
3,baseline_optimized,24.8%,0.1%,medium,✅
4,aggressive_learning,21.1%,0.2%,medium,✅
5,fast_convergence,20.2%,1.7%,medium,✅
6,large_model_regularized,18.3%,0.0%,large,✅
-,ctc_alignment_free,FAILED,FAILED,-,❌


2025-06-24 13:24:59,971 - INFO - 💾 Results saved to colab_hyperparameter_results_20250624_132459.json



🎉 Hyperparameter tuning completed!
🏆 Best result: conservative_small
📊 Character accuracy: 29.3%
📊 Sequence accuracy: 5.4%


## 🎯 Run Specific Experiments

If you want to run only certain experiments:

In [ ]:
# Run specific experiments
experiments_to_run = [
    'conservative_small',
    'baseline_optimized',
    'focal_loss_experiment'
]

print(f"🎯 Running selected experiments: {experiments_to_run}")

selective_tuner = ColabHyperparameterTuner()
selective_tuner.run_experiments(experiments_to_run)

# Save results
results_file = selective_tuner.save_results()
print(f"\n💾 Results saved: {results_file}")

## 📥 Download Results

In [ ]:
# Download results and model checkpoints
from google.colab import files
import glob
import zipfile

# Find all result files
result_files = glob.glob("colab_hyperparameter_results_*.json")
log_files = glob.glob("*.log")
checkpoint_dirs = glob.glob("training_output/*")

print("📁 Available files for download:")
print("\n📊 Results:")
for f in result_files:
    print(f"  - {f}")

print("\n📝 Logs:")
for f in log_files:
    print(f"  - {f}")

print("\n🏗️ Model checkpoints:")
for d in checkpoint_dirs:
    print(f"  - {d}/")

# Download the latest results file
if result_files:
    latest_results = sorted(result_files)[-1]
    print(f"\n📥 Downloading latest results: {latest_results}")
    files.download(latest_results)

# Create and download a zip of all training outputs
if checkpoint_dirs:
    zip_filename = f"training_outputs_{datetime.now().strftime('%Y%m%d_%H%M%S')}.zip"
    
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        # Add result files
        for f in result_files:
            zipf.write(f)
        
        # Add checkpoints (best models only to save space)
        for checkpoint_dir in checkpoint_dirs:
            best_model = f"{checkpoint_dir}/checkpoints/best_model.pth"
            if os.path.exists(best_model):
                zipf.write(best_model)
    
    print(f"\n📦 Downloading complete training package: {zip_filename}")
    files.download(zip_filename)

print("\n✅ Download completed!")

## 📊 TensorBoard Visualization (Optional)

In [ ]:
# Load TensorBoard in Colab
%load_ext tensorboard

# Launch TensorBoard
%tensorboard --logdir training_output

print("📊 TensorBoard launched! You can see training curves above.")
print("💡 Refresh the TensorBoard cell if it doesn't load immediately.")

## 📋 Summary & Next Steps

After running the experiments:

1. **Download your results** using the download cell above
2. **Analyze the results** - check which configuration performed best
3. **Run full training** with the best configuration
4. **Fine-tune** hyperparameters around the best performing range

### Expected Performance:
- **Best configuration**: conservative_small
- **Expected character accuracy**: 40-60% (with full epochs)
- **Target**: 85% character accuracy

### GPU Benefits:
- **~10x faster** training compared to CPU
- **Mixed precision** training for additional speedup
- **Larger batch sizes** possible with GPU memory

🎉 **Congratulations on running hyperparameter tuning on Colab!**
